# Importing Libraries
in this part we will install all the necessary libraries on command prompt and then import the necessary functions from those libraries. 

In [1]:
# importing all the necessary libraries
import pandas as pd

# step 1: preprocessing
from sklearn.impute import SimpleImputer # import some strategic imputer to fill in any missing values using mean
from sklearn.preprocessing import MinMaxScaler # scale all the values to one range to avoid any biasness (this bias is seen in mostly naive bayes and knn etc)
from sklearn.impute import KNNImputer # import some strategic imputer to fill missing values using KNN (finds the nearest neighbour and fills it with that value)

# step 2: data division
from sklearn.model_selection import train_test_split # to divide the code into train/test using a specific percentage or with/without replacement

# step 3: decision trees creation and tuning
from sklearn.tree import DecisionTreeClassifier # to intialize a decision tree maker, its hyper parameters can be defined or set by a grid
from sklearn.model_selection import GridSearchCV # to set different hyper parameters for creating a decision tree

# step 4: displaying accuracy
from sklearn.metrics import roc_auc_score # to display the accuracy of our tree

In [2]:
# use this block to install any libraries not on the system
# !pip install pandas
# !pip install sklearn
# python -m pip install scikit-learn lightgbm xgboost catboost

# Data Loading
data shall be loaded into variables as data sets using pandas and csv readers. they will be checked to see if they are loaded properly and will be loaded as 2 sets: train and test as per given in the kaggle data

In [3]:
# lets load the training data set
train_data = pd.read_csv(r"D:\Users\DELL\OneDrive - Institute of Business Administration\IBA\sem5\machine learning\ipynb notebooks\challenger1\iml-fall-2024-challenge-1\train_set.csv")

In [ ]:
train_data.head() # lets also check it by getting the first few rows of the data, there should be x1 - x78 and one target variable Y

In [5]:
# lets load the test data
test_data = pd.read_csv(r"D:\Users\DELL\OneDrive - Institute of Business Administration\IBA\sem5\machine learning\ipynb notebooks\challenger1\iml-fall-2024-challenge-1\test_set.csv")

In [ ]:
test_data.head() # check if the data has been loaded by getting the first 5 rows - there should be x1 - x78 and no target variable Y as this is test data

# Data Preprocessing
before we start processing this data and using algorithms, we will fix this data first, this is called data preprocessing

## Conversion of Categorical to Numerical
First we will convert categorical data to numerical data by doing one hot encoding, which turns it into binary variables

In [ ]:
pd.get_dummies(train_data) # this line will convert the train_data to one hot encoding but it will only display the result and not save it

In [8]:
# we can see that there is no change in the number of columns meaning there is no categorical data. but for the sake of running the program. we must perform the preprocessing therefore we shall re-run the one hot encoding and save it somewhere
train_data_processed = pd.get_dummies(train_data)

In [9]:
# now we shall do the same on the test data so that we maintain the rules over all data
test_data_processed = pd.get_dummies(test_data)

## Data Splitting - festures and targets
the data in train_data set is of x1 - x78 columns (79 variables) and one target variable (Y). we must split that data so that we can perform data preprocessing on the features variables (will be referred to as X).

In [ ]:
# so in X, it is ALL the columns EXCEPT the last column known as 'Y' (we can confirm this using the train_data.head() we did earlier) so we must get all columns and DROP only the 'y' column
X = train_data_processed.drop(columns=['Y'])
X # lets display X and see what it is now

In [ ]:
# so as per our X output, we can see that number of columns in train_data is 79 and number of columns in X is 78 meaning we have successfully performed our removal of target variable
# now to get the target variable alone, we can just get it alone,
Y = train_data_processed['Y']
Y # lets see what it is

In [14]:
# as per our Y output, we can see it is of one column and 246k rows which means we have successfully extracted the target variable column

## Data Imputation 
many cells in our data may be empty - we must fill these cells with data. we have multiple options to deal with them:
- we remove the entire rows (Case 1)
- we fill the cells with the average of the column (Case 2)
- we fill the cells based on KNN imputation (nearest neighbour) (Case 3)

In [ ]:
# in this case, lets remove the entire rows that have NaN values. before saving the removed rows data set, lets first run it and display it to see the outcome, then we shall save in X
X.dropna(axis=0)

In [10]:
# so we originally had 246122 rows and now after removing empty cell rows we have 239650 rows which is a 6472 rows difference. as our first try, lets work with it. lets assign this data set in place of X
# X = X.dropna(axis=0)
# X
train_data_processed = train_data_processed.dropna(axis=0)

## Data Scaling
some columns may be very large then other columns when compared. it would not affect at the moment as we are using decision trees, but to maintain a fair enviroment, we shall perform scaling on every run.
there are two types of scaling: 
- min max scaling (also known as normalization)
- standardisation 

In [ ]:
# in this case we shall perform min max scaling. to do that, we must use our MinMaxScaler that we have imported above
scaler = MinMaxScaler()
# now we must use this scaler to scale X
scaler.fit_transform(X)

In [14]:
# our output shows us that every value in the array is between 0 and 1. thus lets save this value on X
X = scaler.fit_transform(X)

# now we must do the same on our test_data set
test_data_processed = scaler.fit_transform(test_data_processed)

## Data Splitting - train and validate
now our test_data set is of rows with NO target variable whereas the train_data set is WITH target variable. our rules in machine learning is that we must train half or 70% of the data and then we must check its accuracy using the remaining half or 30% of the data - we can only check accuracy IF we have the answers i.e. the target variable. 
So, what we need to do is, is split the train_data set into 2, by a 70% and 30% ratio. we train the model using the 70% and then test the model using the 30% and then use that model to predict the test_data set.

In [15]:
trainX, testX, trainY, testY = train_test_split(X, Y, test_size=0.3)

### ERROR SOLUTION
one error approached was that we had first splitted the data into features and target variables (X, Y) and then dropped all the NaN rows from X which made X and Y have differnet number of rows. after trying to "index" and match these rows, nothing was working. So a solution was found to restart the ipynb and first remove all the NaN rows from the train_data_processed and THEN split it into X, Y which resulted in same rows and this code line worked. 

# Model Algorithm Running
now we shall begin running our model

In [169]:
# lets intialize our decision tree with some parameters
decisiontree = DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_split=15, max_features=60)

In [ ]:
# now lets fit this model onto our data
decisiontree.fit(trainX, trainY)

In [171]:
# after fitting, lets predict our testX and analyse the accuracy
prediction = decisiontree.predict(testX)

In [172]:
# now lets calculate the ROC AUC score according to this prediction
roc_score = roc_auc_score(testY, prediction)

In [ ]:
print("roc score = ", roc_score)

In [ ]:
decisiontree.fit(X, Y)

In [175]:
test_prediction = decisiontree.predict_proba(test_data_processed)

In [176]:
test_prediction=test_prediction[:, 1]

In [ ]:
print(test_prediction)

In [ ]:
sample_data = pd.read_csv(r"D:\Users\DELL\OneDrive - Institute of Business Administration\IBA\sem5\machine learning\ipynb notebooks\challenger1\iml-fall-2024-challenge-1\sample_submission.csv")
sample_data


In [ ]:
sample_data['Y'] = test_prediction
sample_data

In [ ]:
sample_data.to_csv(r"D:\Users\DELL\OneDrive - Institute of Business Administration\IBA\sem5\machine learning\ipynb notebooks\challenger1\iml-fall-2024-challenge-1\sample_submission.csv", index=False)

sample_data

In [ ]:
checking = pd.read_csv(r"D:\Users\DELL\OneDrive - Institute of Business Administration\IBA\sem5\machine learning\ipynb notebooks\challenger1\iml-fall-2024-challenge-1\sample_submission.csv")
checking